In [10]:
# import os
# import cv2
# import numpy as np
# from torch.utils.data import Dataset
# from sklearn.model_selection import train_test_split


# class AFLWDataset(Dataset):
#     def __init__(self, image_paths, annotation_paths):
#         self.image_paths = image_paths
#         self.annotation_paths = annotation_paths

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         img_path = self.image_paths[idx]
#         pts_path = self.annotation_paths[idx]

#         image = cv2.imread(img_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         landmarks = self.read_pts(pts_path)

#         return image, landmarks, img_path

#     def read_pts(self, pts_path):
#         coords = []
#         with open(pts_path, 'r') as f:
#             for line in f:
#                 parts = line.strip().split()
#                 if len(parts) == 3:
#                     try:
#                         x, y = float(parts[0]), float(parts[1])
#                         vis = parts[2].lower() == 'true'
#                         coords.append([x, y, vis])
#                     except ValueError:
#                         continue
#         return np.array(coords).reshape(-1, 3)


In [11]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import torch
import cv2

class AFLWDataset(Dataset):
    def __init__(self, image_paths, pts_paths, image_size=(224, 224)):
        self.image_paths = image_paths
        self.pts_paths = pts_paths
        self.image_size = image_size  # (H, W)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        pts_path = self.pts_paths[idx]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Original size
        h0, w0 = image.shape[:2]

        # Resize image
        image = cv2.resize(image, self.image_size)

        # Load landmarks and scale to new image size
        landmarks = self.read_pts(pts_path)
        landmarks = np.array(landmarks).reshape(-1, 3)
        landmarks = landmarks[:, :2]  # Drop the visibility flag if present

        # Scale landmarks
        landmarks[:, 0] *= self.image_size[1] / w0
        landmarks[:, 1] *= self.image_size[0] / h0

        # Convert to tensor
        image = torch.tensor(image).permute(2, 0, 1).float() / 255.0  # [C,H,W]
        landmarks = torch.tensor(landmarks).float()  # [N,2]

        return image, landmarks

    def read_pts(self, path):
        with open(path, 'r') as f:
            lines = f.readlines()
        return [list(map(float, line.strip().split())) for line in lines if line.strip()]


In [12]:
from sklearn.model_selection import train_test_split

def collect_aflw_paths(image_root, annotation_root):
    image_paths = []
    annotation_paths = []

    for style_folder in sorted(os.listdir(image_root)):
        style_path = os.path.join(image_root, style_folder)
        if not os.path.isdir(style_path):
            continue

        for folder in sorted(os.listdir(style_path)):
            img_folder = os.path.join(style_path, folder)
            if not os.path.isdir(img_folder):
                continue

            for fname in os.listdir(img_folder):
                if fname.endswith(".jpg"):
                    img_path = os.path.join(img_folder, fname)
                    ann_subdir = os.path.join(annotation_root, folder)
                    if not os.path.exists(ann_subdir):
                        continue

                    pts_candidates = [f for f in os.listdir(ann_subdir) if f.startswith(fname[:-4])]
                    if pts_candidates:
                        pts_path = os.path.join(ann_subdir, pts_candidates[0])
                        image_paths.append(img_path)
                        annotation_paths.append(pts_path)

    return image_paths, annotation_paths


# ✅ Set your dataset path
image_root = '/media/cse/ML/Miju_work/Dataset/AFLW-Style/AFLW-Style'
annotation_root = os.path.join(image_root, 'annotation')

# 📦 Gather all image/annotation paths
all_images, all_annotations = collect_aflw_paths(image_root, annotation_root)

# 🔀 Split into training and test sets
train_imgs, test_imgs, train_anns, test_anns = train_test_split(
    all_images, all_annotations, test_size=0.2, random_state=42
)

# ✅ Create dataset objects
train_dataset = AFLWDataset(train_imgs, train_anns)
test_dataset = AFLWDataset(test_imgs, test_anns)


In [13]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [14]:
import torch.nn as nn

class LandmarkModel(nn.Module):
    def __init__(self, num_points=19):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 1024), nn.ReLU(),
            nn.Linear(1024, num_points * 2)  # (x,y) for each point
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [15]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for imgs, targets in dataloader:
        imgs, targets = imgs.to(device), targets.to(device)
        optimizer.zero_grad()
        preds = model(imgs)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)


In [16]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model setup
model = LandmarkModel()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Train
for epoch in range(10):
    loss = train(model, train_loader, optimizer, criterion, device)
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}")


ValueError: could not convert string to float: 'True'